<a href="https://colab.research.google.com/github/adwitaarora/ai-hw-lab/blob/main/AIHWPractical2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

params = {
    'dataset': 'sem_eval_2018_task_1',
}

response = requests.get('https://datasets-server.huggingface.co/splits', params=params)

In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import list_datasets, load_dataset

In [ ]:
dataset = load_dataset('sem_eval_2018_task_1', 'subtask5.english')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

Dataset sem_eval_2018_task_1 downloaded and prepared to /root/.cache/huggingface/datasets/sem_eval_2018_task_1/subtask5.english/1.1.0/a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset_train = dataset["train"]

In [ ]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

In [ ]:
df = dataset_train.data.to_pandas()

In [ ]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.0 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [ ]:
df.columns

Index(['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy',
       'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
      dtype='object')

In [ ]:
df = df.drop(['pessimism', 'surprise', 'trust'], axis = 1)

In [ ]:
def change(x):
  if x == True:
    return 1
  else :
    return 0
emotions = ['anger','anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'sadness']
for em in emotions:
  df[em] = df[em].apply(lambda x : change(x))

In [ ]:
df

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,sadness
0,2017-En-21441,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0
1,2017-En-31535,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0
2,2017-En-21068,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0
3,2017-En-31436,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0
4,2017-En-22195,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
6833,2017-En-21383,@nicky57672 Hi! We are working towards your hi...,0,0,0,0,0,0,0,0
6834,2017-En-41441,@andreamitchell said @berniesanders not only d...,0,1,0,0,0,0,0,0
6835,2017-En-10886,@isthataspider @dhodgs i will fight this guy! ...,1,0,1,0,0,0,0,0
6836,2017-En-40662,i wonder how a guy can broke his penis while h...,0,0,0,0,0,0,0,0


In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

In [ ]:
def b_tp(preds, labels):
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [ ]:
def preprocessing(input_text, tokenizer):
    return tokenizer.encode_plus(
                          input_text,
                          add_special_tokens = True,
                          max_length = 32,
                          pad_to_max_length = True,
                          return_attention_mask = True,
                          return_tensors = 'pt'
                    )

def bert(text, labels):
  token_id = []
  attention_masks = []


  for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids']) 
    attention_masks.append(encoding_dict['attention_mask'])


  token_id = torch.cat(token_id, dim = 0)
  attention_masks = torch.cat(attention_masks, dim = 0)
  labels = torch.tensor(labels)
  val_ratio = 0.2
  batch_size = 16

  train_idx, val_idx = train_test_split(
      np.arange(len(labels)),
      test_size = val_ratio,
      shuffle = True,
      stratify = labels)

  train_set = TensorDataset(token_id[train_idx], 
                            attention_masks[train_idx], 
                            labels[train_idx])

  val_set = TensorDataset(token_id[val_idx], 
                          attention_masks[val_idx], 
                          labels[val_idx])

  # Prepare DataLoader
  train_dataloader = DataLoader(
              train_set,
              sampler = RandomSampler(train_set),
              batch_size = batch_size
          )

  validation_dataloader = DataLoader(
              val_set,
              sampler = SequentialSampler(val_set),
              batch_size = batch_size
          )
  model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
  )

  optimizer = torch.optim.AdamW(model.parameters(), 
                                lr = 5e-5,
                                eps = 1e-08
                                )

  # Run on GPU
  model.cuda()
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  epochs = 2

  for _ in trange(epochs, desc = 'Epoch'):
      
      
      model.train()
      
      tr_loss = 0
      nb_tr_examples, nb_tr_steps = 0, 0

      for step, batch in enumerate(train_dataloader):
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          optimizer.zero_grad()
        
          train_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask, 
                              labels = b_labels)
        
          train_output.loss.backward()
          optimizer.step()
        
          tr_loss += train_output.loss.item()
          nb_tr_examples += b_input_ids.size(0)
          nb_tr_steps += 1

    
      model.eval()

    
      val_accuracy = []
      val_precision = []
      val_recall = []
      val_specificity = []

      for batch in validation_dataloader:
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          with torch.no_grad():
          
            eval_output = model(b_input_ids, 
                                token_type_ids = None, 
                                attention_mask = b_input_mask)
          logits = eval_output.logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()

          b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
          val_accuracy.append(b_accuracy)
          if b_precision != 'nan': val_precision.append(b_precision)
          if b_recall != 'nan': val_recall.append(b_recall)
          if b_specificity != 'nan': val_specificity.append(b_specificity)

      print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
      print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
      print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
      print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
      print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')
  return model

  # test_ids = []
  # test_attention_mask = []

  # encoding = preprocessing(new_sentence, tokenizer)

  # test_ids.append(encoding['input_ids'])
  # test_attention_mask.append(encoding['attention_mask'])
  # test_ids = torch.cat(test_ids, dim = 0)
  # test_attention_mask = torch.cat(test_attention_mask, dim = 0)

  # with torch.no_grad():
  #   output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

  # prediction = f'{c_class}' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Not Detected'

  # print('Input Sentence: ', new_sentence)
  # print('Predicted Class: ', prediction)


In [ ]:
def prediction(model, new_sentence, c_class):
    test_ids = []
    test_attention_mask = []

    encoding = preprocessing(new_sentence, tokenizer)

    test_ids.append(encoding['input_ids'])
    test_attention_mask.append(encoding['attention_mask'])
    test_ids = torch.cat(test_ids, dim = 0)
    test_attention_mask = torch.cat(test_attention_mask, dim = 0)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    with torch.no_grad():
      output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

    prediction = f'{c_class}' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Not Detected'

    # print('Input Sentence: ', new_sentence)
    # print('Predicted Class: ', prediction)
    return prediction

In [ ]:
text = df.Tweet.values
labels = df.anger.values
model_anger = bert(text, labels)


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.4617
	 - Validation Accuracy: 0.8154
	 - Validation Precision: 0.7488
	 - Validation Recall: 0.7600
	 - Validation Specificity: 0.8466



Epoch: 100%|██████████| 2/2 [01:27<00:00, 43.54s/it]


	 - Train loss: 0.2876
	 - Validation Accuracy: 0.8154
	 - Validation Precision: 0.7267
	 - Validation Recall: 0.8190
	 - Validation Specificity: 0.8133



In [ ]:
pred = prediction(model_anger, "this is good","anger")

Input Sentence:  this is good
Predicted Class:  Not Detected


In [ ]:
text = df.Tweet.values
labels = df.anticipation.values
model_anticipation = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.4133
	 - Validation Accuracy: 0.8576
	 - Validation Precision: NaN
	 - Validation Recall: 0.0000
	 - Validation Specificity: 1.0000



Epoch: 100%|██████████| 2/2 [01:36<00:00, 48.18s/it]


	 - Train loss: 0.3689
	 - Validation Accuracy: 0.8474
	 - Validation Precision: 0.3929
	 - Validation Recall: 0.1621
	 - Validation Specificity: 0.9624



In [ ]:
text = df.Tweet.values
labels = df.disgust.values
model_disgust = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.5121
	 - Validation Accuracy: 0.7769
	 - Validation Precision: 0.6858
	 - Validation Recall: 0.7754
	 - Validation Specificity: 0.7722



Epoch: 100%|██████████| 2/2 [01:27<00:00, 43.69s/it]


	 - Train loss: 0.3307
	 - Validation Accuracy: 0.7776
	 - Validation Precision: 0.7042
	 - Validation Recall: 0.7260
	 - Validation Specificity: 0.8032



In [ ]:
text = df.Tweet.values
labels = df.fear.values
model_fear = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.3316
	 - Validation Accuracy: 0.8990
	 - Validation Precision: 0.7831
	 - Validation Recall: 0.6455
	 - Validation Specificity: 0.9595



Epoch: 100%|██████████| 2/2 [01:27<00:00, 43.69s/it]


	 - Train loss: 0.2295
	 - Validation Accuracy: 0.8932
	 - Validation Precision: 0.7573
	 - Validation Recall: 0.6387
	 - Validation Specificity: 0.9544



In [ ]:
text = df.Tweet.values
labels = df.joy.values
model_joy = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.4315
	 - Validation Accuracy: 0.8517
	 - Validation Precision: 0.8415
	 - Validation Recall: 0.7162
	 - Validation Specificity: 0.9214



Epoch: 100%|██████████| 2/2 [01:27<00:00, 43.85s/it]


	 - Train loss: 0.2377
	 - Validation Accuracy: 0.8379
	 - Validation Precision: 0.7585
	 - Validation Recall: 0.7985
	 - Validation Specificity: 0.8572



In [ ]:
text = df.Tweet.values
labels = df.love.values
model_love = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.2815
	 - Validation Accuracy: 0.9077
	 - Validation Precision: 0.5347
	 - Validation Recall: 0.5343
	 - Validation Specificity: 0.9481



Epoch: 100%|██████████| 2/2 [01:27<00:00, 43.76s/it]


	 - Train loss: 0.1879
	 - Validation Accuracy: 0.9106
	 - Validation Precision: 0.5709
	 - Validation Recall: 0.5026
	 - Validation Specificity: 0.9524



In [ ]:
text = df.Tweet.values
labels = df.optimism.values
model_optimism = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.4675
	 - Validation Accuracy: 0.8176
	 - Validation Precision: 0.6866
	 - Validation Recall: 0.6941
	 - Validation Specificity: 0.8683



Epoch: 100%|██████████| 2/2 [01:36<00:00, 48.26s/it]


	 - Train loss: 0.3036
	 - Validation Accuracy: 0.7776
	 - Validation Precision: 0.5889
	 - Validation Recall: 0.7877
	 - Validation Specificity: 0.7731



In [ ]:
text = df.Tweet.values
labels = df.sadness.values
model_sadness = bert(text, labels)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceCla


	 - Train loss: 0.5386
	 - Validation Accuracy: 0.7863
	 - Validation Precision: 0.6562
	 - Validation Recall: 0.5490
	 - Validation Specificity: 0.8754



Epoch: 100%|██████████| 2/2 [01:30<00:00, 45.17s/it]


	 - Train loss: 0.4021
	 - Validation Accuracy: 0.7703
	 - Validation Precision: 0.6713
	 - Validation Recall: 0.4343
	 - Validation Specificity: 0.9040



In [ ]:
model_to_emotion = {model_anger : 'anger', model_anticipation : 'anticipation', model_disgust : 'disgust', model_fear : 'fear', model_joy : 'joy', model_love : 'love', model_optimism : 'optimism', model_sadness : 'sandess'}

In [ ]:
def emo(sentence):
  emotions = []
  for model in model_to_emotion.keys():
    pred = prediction(model, sentence, model_to_emotion[model])
    if pred != 'Not Detected':
      emotions.append(pred)
  for emo in emotions:
    print(emo)

In [ ]:
sentence = "this is such a good movie"
emo(sentence)

joy
love
optimism


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
import torch
for model in model_to_emotion.keys():
  torch.save(model, '/content/drive/MyDrive/models/model')
  # saved_model = torch.load('path/to/model')

RuntimeError: ignored

In [ ]:
nh fgjfjfnjfghfhgcjvgv bn vnb, .from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
